In [ ]:
!pip install gradio

In [ ]:
!pip install bitsandbytes
!pip install trainsformers
!pip install peft
!pip install accelerate
!pip install datasets

In [1]:
# tokenizer
from transformers import AutoTokenizer
model_id = "beomi/KoAlpaca-Polyglot-5.8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,)
# set pad token - to avoid error while training
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
!git clone https://github.com/mnbv7581/document_summary.git
%cd document_summary
!pip install -e .

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import prepare_model_for_kbit_training


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("/content/gdrive/MyDrive/Colab Notebooks/checkpoint-18500", quantization_config=bnb_config, device_map={"":0})

# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
#Setting the Pretraining_tp to 1 ensures we are using the Linear Layers to the max computation possible

model.config.use_cache = True
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
# gradio library를 가져온다. gr로 줄여서 칭한다.
import gradio as gr
from src.train import Trainer

llm_summerizer = Trainer()
# 챗봇에 채팅이 입력되면 이 함수를 호출합니다.
# message는 유저의 채팅 메시지, history는 채팅 기록, additional_input_info는 additional_inputs안 블록의 정보를 받습니다.
def response(message, history, additional_input_info):
  
    summary = llm_summerizer.test(message,model,tokenizer)

    return summary

gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="url을 입력해주세요", container=False, scale=7),
        title="뉴스기사를 요약해주는 AI 입니다.",
        description="네이버 뉴스 url을 입력하면 요약해주는 챗봇서비스 입니다",
        theme="soft",
        examples=[["https://n.news.naver.com/article/661/0000041753?type=main"], ["https://n.news.naver.com/article/015/0005002147?cds=news_media_pc&type=editn"], ["https://n.news.naver.com/article/024/0000089905?cds=news_media_pc&type=editn"]],
        retry_btn="다시보내기 ↩",
        undo_btn="이전챗 삭제 ↺",
        clear_btn="전챗 삭제 ✄"
).launch()